In [153]:
%matplotlib inline
import matplotlib
import seaborn as sns
import pprint
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

In [8]:
import requests
years=range(2010,2017)
url = "https://en.wikipedia.org/wiki/List_of_American_films_of_"
responses = [requests.get(url+str(year)) for year in years ]
print responses[0].url
print responses[0].text[:1000] + "..."
print len(responses)

https://en.wikipedia.org/wiki/List_of_American_films_of_2010
<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>List of American films of 2010 - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_American_films_of_2010","wgTitle":"List of American films of 2010","wgCurRevisionId":755839865,"wgRevisionId":755839865,"wgArticleId":25636852,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Incomplete film lists","Commons category with local link same as on Wikidata","2010 in American cinema","Lists of 2010 films by country or language","Lists of American films by year"],"wgBreakFrames":false,"wgPageC

In [12]:
from bs4 import BeautifulSoup
soups = [BeautifulSoup(response.text) for response in responses]
print soups[6].head()

[<meta charset="utf-8"/>, <title>List of American films of 2016 - Wikipedia</title>, <script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>, <script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_American_films_of_2016","wgTitle":"List of American films of 2016","wgCurRevisionId":760866101,"wgRevisionId":760866101,"wgArticleId":45452962,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Commons category with local link same as on Wikidata","Lists of American films by year","Lists of 2016 films by country or language","2016 in American cinema"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","w

In [37]:
soups[2].select('table.wikitable')[0].select('th')[0].text

u'Title'

In [75]:
tables = [ soup.select('table.wikitable') for soup in soups]

In [81]:
len(tables)

7

In [82]:
columns = [table[0].select('th') for table in tables]

In [83]:
features = zip(years,[[c.text for c in col] for col in columns])

In [84]:
features

[(2010,
  [u'Title',
   u'Director',
   u'Cast(Subject of documentary)',
   u'Genre',
   u'Notes']),
 (2011,
  [u'Title',
   u'Director',
   u'Cast (Subject of documentary)',
   u'Genre',
   u'Notes']),
 (2012,
  [u'Title',
   u'Director',
   u'Cast (Subject of documentary)',
   u'Genre',
   u'Notes',
   u'Release date']),
 (2013,
  [u'Title',
   u'Director',
   u'Cast (Subject of documentary)',
   u'Genre',
   u'Notes',
   u'Release date',
   u'Budget/BoxOffice $million']),
 (2014,
  [u'Opening', u'Title', u'Director', u'Cast', u'Genre', u'Notes', u'Ref.']),
 (2015,
  [u'Opening',
   u'Title',
   u'Director',
   u'Cast',
   u'Genre',
   u'Notes',
   u'Ref.',
   u'J\nA\nN\nU\nA\nR\nY',
   u'2',
   u'9',
   u'27']),
 (2016,
  [u'Opening', u'Title', u'Director', u'Cast', u'Genre', u'Notes', u'Ref.'])]

In [109]:
vintages = [[t.select('td') for t in table ]for table in tables]

In [207]:
data={'2010':[] , '2011':[], '2012':[], '2013':[], '2014':[], '2015':[], '2016':[]}
for year,vintage in zip(years,vintages):
    for releases in vintage:
        credits =[release.text for release in releases]
        data[str(year)].append(credits)
    

In [200]:
data['2013'][0]

[u'2 Guns',
 u'Baltasar Korm\xe1kur',
 u'Denzel Washington, Mark Wahlberg, Paula Patton, Bill Paxton, Fred Ward, James Marsden, Edward James Olmos',
 u'Action comedy',
 u'Universal Pictures\nBased on the Boom! Studios graphic novels by Steven Grant',
 u'000000002017-08-02-0000August 2',
 u'61 / 132',
 u'12 Rounds 2: Reloaded',
 u'Roel Rein\xe9',
 u'Randy Orton, Brian Markinson, Venus Terzo',
 u'Action',
 u'20th Century Fox, WWE Studios\nDirect-to-video',
 u'000000002017-06-04-0000June 4',
 u'N/A',
 u'12 Years a Slave',
 u'Steve McQueen',
 u"Chiwetel Ejiofor, Michael Fassbender, Benedict Cumberbatch, Paul Dano, Paul Giamatti, Lupita Nyong'o, Sarah Paulson, Brad Pitt, Alfre Woodard",
 u'Drama',
 u'Fox Searchlight Pictures\nBased on the autobiography of the same name by Solomon Northup',
 u'000000002017-10-18-0000October 18',
 u'20 / 188',
 u'20 Feet from Stardom',
 u'Morgan Neville',
 u'A look at the lives of background singers.',
 u'Documentary',
 u'The Weinstein Company',
 u'0000000020

In [157]:
movies = { '2010':[], '2011':[], '2012':[], '2013':[], '2014':[], '2015':[], '2016':[]}

In [158]:
for data in data['2010']:
    while data:
        try:
            notes = data.pop()
            genre = data.pop()
            cast= data.pop()
            director = data.pop()
            title= data.pop()
            movies['2010'].append({'title':title, 'director':director, 'cast':cast, 'genre': genre, 'notes': notes})
        except IndexError:
            continue

In [164]:
for data in data['2011']:
    while data:
        try:
            notes = data.pop()
            genre = data.pop()
            cast= data.pop()
            director = data.pop()
            title= data.pop()
            movies['2011'].append({'title':title, 'director':director, 'cast':cast, 'genre': genre, 'notes': notes})
        except IndexError:
            continue

In [186]:
for data in data['2012']:
    while data:
        try:
            
            if title!= u'Battery, TheThe Battery':
                release_date = data.pop()
            else:
                release_date = ''
            notes = data.pop()
            genre = data.pop()
            cast= data.pop()
            director = data.pop()
            title= data.pop()              
            movies['2012'].append({'title':title, 'director':director, 'cast':cast, 'genre': genre, 'notes': notes, 'release_date':release_date})
                
        except IndexError:
            continue

In [224]:
movies['2012'][-1]

{'cast': u'Sara Malakul Lane, Jesus Guevara, Steve Hanks, Carl Donelson, Laura Ramos',
 'director': u'Jared Cohn',
 'genre': u'Horror',
 'notes': u'Asylum, TheThe Asylum',
 'release_date': u'000000002017-12-04-0000December 4',
 'title': u' 12/12/12'}

In [208]:
for data in data['2013']:
    while data:
        try:
            if title!= u'Frances Ha':
                budget_vs_boxoffice = data.pop()
            else:
                budget_vs_boxoffice = ''
            release_date = data.pop()
            notes = data.pop()
            genre = data.pop()
            cast= data.pop()
            director = data.pop()
            title= data.pop()              
            movies['2013'].append({'title':title, 'director':director, 'cast':cast, 
                                   'genre': genre, 'notes': notes, 'release_date':release_date,
                                  'budget_vs_boxoffice': budget_vs_boxoffice})
                
        except IndexError:
            continue

In [222]:
movies['2013'][0]

{'budget_vs_boxoffice': u'N/A',
 'cast': u'Danny Trejo, Martin Copping, Clare Niederpruem',
 'director': u'Kevin King',
 'genre': u'Thriller',
 'notes': u'Highland Film Group, Direct-to-video',
 'release_date': u'000000002017-07-18-0000July 18',
 'title': u'Zombie Hunter'}